In [1]:
%load_ext nb_black
# !pip install nb_black
import warnings

warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics as m
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE  # doctest: +NORMALIZE_WHITESPACE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn import model_selection
from mlxtend.classifier import StackingClassifier
from thundersvm import SVC as svmgpu
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE, SMOTENC  # doctest: +NORMALIZE_WHITESPACE
from multiscorer import MultiScorer
from sklearn.model_selection import cross_val_score
from numpy import average
import xgboost as xgb
from sklearn import metrics as m
import itertools

Using TensorFlow backend.


<IPython.core.display.Javascript object>

In [3]:
randomseed = 7
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

In [4]:
data = pd.read_csv("test-temp.csv")
originalFeatures = [
    "FPG",
    "HbA1c",
    "Triglyceride",
    "BMI",
    "r-GTP gamma",
    "Uric acid",
    "Smoking",
    "Drinking",
    "Physical activity",
    "Family history",
    "SEX",
    "AGE",
]

<IPython.core.display.Javascript object>

In [5]:
data = data.sort_values(by="Model")
data

,Features,Accuracy,Precision,Recall,F1-score,CV_Accuracy,CV_Accuracy_std,Model
41,"['FPG', 'HbA1c', 'Smoking', 'Family history', ...",0.747000,0.756000,0.747000,0.750000,0.732000,0.040000,RF
31,"['FPG', 'HbA1c', 'BMI', 'Uric acid', 'Smoking'...",0.742000,0.752000,0.742000,0.745000,0.746000,0.038000,RF
33,"['FPG', 'HbA1c', 'BMI', 'Smoking', 'Physical a...",0.740000,0.747000,0.740000,0.743000,0.743000,0.039000,RF
38,"['FPG', 'HbA1c', 'BMI', 'Smoking', 'SEX', 'AGE']",0.750000,0.758000,0.750000,0.753000,0.743000,0.039000,RF
39,"['FPG', 'HbA1c', 'Smoking', 'Physical activity...",0.748000,0.756000,0.748000,0.751000,0.759000,0.017000,RF
...,...,...,...,...,...,...,...,...
48,"['FPG', 'HbA1c', 'Smoking', 'Physical activity...",0.747000,0.747000,0.747000,0.747000,0.737000,0.015000,XGB
40,"['FPG', 'HbA1c', 'Smoking', 'Physical activity...",0.747000,0.748000,0.747000,0.747000,0.736000,0.015000,XGB
65,"['FPG', 'HbA1c', 'r-GTP gamma', 'Drinking']",0.742000,0.742000,0.742000,0.741000,0.738000,0.014000,XGB
47,"['FPG', 'HbA1c', 'Uric acid', 'Physical activi...",0.747000,0.747000,0.747000,0.747000,0.736000,0.014000,XGB


<IPython.core.display.Javascript object>

In [6]:
for i in range(data.shape[0]):
    temp = list(
        str(data.iloc[i, 0])
        .replace("[", "")
        .replace("]", "")
        .replace("'", "")
        .split(",")
    )
    temp_changed = []
    for j in range(len(temp)):
        temp_changed.append(originalFeatures.index(temp[j].strip()))
    data.iloc[i, 0] = str(temp_changed)
data

,Features,Accuracy,Precision,Recall,F1-score,CV_Accuracy,CV_Accuracy_std,Model
41,"[0, 1, 6, 9, 10, 11]",0.747000,0.756000,0.747000,0.750000,0.732000,0.040000,RF
31,"[0, 1, 3, 5, 6, 10, 11]",0.742000,0.752000,0.742000,0.745000,0.746000,0.038000,RF
33,"[0, 1, 3, 6, 8, 10, 11]",0.740000,0.747000,0.740000,0.743000,0.743000,0.039000,RF
38,"[0, 1, 3, 6, 10, 11]",0.750000,0.758000,0.750000,0.753000,0.743000,0.039000,RF
39,"[0, 1, 6, 8, 9, 10]",0.748000,0.756000,0.748000,0.751000,0.759000,0.017000,RF
...,...,...,...,...,...,...,...,...
48,"[0, 1, 6, 8, 10]",0.747000,0.747000,0.747000,0.747000,0.737000,0.015000,XGB
40,"[0, 1, 6, 8, 9, 10]",0.747000,0.748000,0.747000,0.747000,0.736000,0.015000,XGB
65,"[0, 1, 4, 7]",0.742000,0.742000,0.742000,0.741000,0.738000,0.014000,XGB
47,"[0, 1, 5, 8, 10]",0.747000,0.747000,0.747000,0.747000,0.736000,0.014000,XGB


<IPython.core.display.Javascript object>

In [7]:
# test_list = (
#     (data.iloc[0, 0]).replace("[", "").replace("]", "").replace("'", "").split(",")
# )
# print(test_list)
# test_list = [int(i) for i in test_list]
# test_list

<IPython.core.display.Javascript object>

In [8]:
modelslist = data.copy()
modelslist = modelslist[modelslist.CV_Accuracy > 0.73]
modelslist = modelslist.sort_values(
    by=["Accuracy", "CV_Accuracy"], ascending=False
).reset_index(drop=True)
modelslist.head(25)

,Features,Accuracy,Precision,Recall,F1-score,CV_Accuracy,CV_Accuracy_std,Model
0,"[0, 1, 7, 9, 10]",0.750000,0.755000,0.750000,0.752000,0.760000,0.01600,RF
1,"[0, 1, 2, 3, 4, 6, 10, 11]",0.750000,0.758000,0.750000,0.753000,0.750000,0.03800,RF
2,"[0, 1, 3, 6, 10, 11]",0.750000,0.758000,0.750000,0.753000,0.743000,0.03900,RF
3,"[0, 1, 10]",0.748333,0.747919,0.748333,0.748065,0.733992,0.01145,XGB
4,"[0, 1, 7, 10]",0.748000,0.756000,0.748000,0.751000,0.760000,0.01800,RF
5,"[0, 1, 6, 8, 9, 10]",0.748000,0.756000,0.748000,0.751000,0.759000,0.01700,RF
6,"[0, 1, 3, 4, 6, 9, 10, 11]",0.748000,0.755000,0.748000,0.751000,0.745000,0.03700,RF
7,"[0, 1, 2, 8, 10]",0.747000,0.753000,0.747000,0.749000,0.767000,0.01500,RF
8,"[0, 1, 2, 3, 4, 6, 9, 10, 11]",0.747000,0.755000,0.747000,0.750000,0.751000,0.03700,RF
9,"[0, 1, 3, 4, 5, 10, 11]",0.747000,0.755000,0.747000,0.750000,0.750000,0.03800,RF


<IPython.core.display.Javascript object>

In [9]:
# Rf_list = (
#     data.query("Model=='RF'")
#     .sort_values(by="CV_Accuracy", ascending=False)
#     .reset_index(drop=True)
# )
# xgb_list = (
#     data.query("Model=='XGB'")
#     .sort_values(by="CV_Accuracy", ascending=False)
#     .reset_index(drop=True)
# )

# svm_list = (
#     data.query("Model=='SVM'")
#     .sort_values(by="CV_Accuracy", ascending=False)
#     .reset_index(drop=True)
# ).iloc[:10, :]

# svm_list

<IPython.core.display.Javascript object>

In [10]:
# Rf_list

<IPython.core.display.Javascript object>

 2.Read dataset
 ===

In [11]:
# read the data set
x_original = pd.read_csv("../../../../dataset/XLable_onlyDiabeticRemoved.txt")

conditions = [
    (x_original.L100800 < 100),
    (x_original.L100800 >= 100) & (x_original.L100800 < 126),
    (x_original.L100800 >= 126),
]
choices = [0, 1, 2]
x_original["CLASS"] = np.select(conditions, choices, default=0)
x_original = x_original[
    [
        "Unnamed: 0",
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",  #'CLASS',
        "FIELD_16",
        "FIELD_23",
        "FIELD_15",
        "FIELD_22",
        "FIELD_17",
        "FIELD_24",
    ]
]

print(x_original.shape)

y_original = pd.read_csv("../../../../dataset/TargetLable_onlyDiabeticRemoved.txt")

conditions = [
    (y_original.L100800 < 100),
    (y_original.L100800 >= 100) & (y_original.L100800 < 126),
    (y_original.L100800 >= 126),
]

choices = [0, 1, 2]
y_original["CLASS"] = np.select(conditions, choices, default=0)

y_original = y_original[["Unnamed: 0", "CLASS"]]

print(y_original.shape)

data = pd.merge(
    x_original, y_original, how="inner", left_on="Unnamed: 0", right_on="Unnamed: 0"
)



# filter the data set
data = data[data.FIELD_16 != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23 != 1]  # exclude people who are on medication for diabetes

data = data[
    data.FIELD_15 != 1
]  # exclude people who are diagnosed for (high blood pressure)
data = data[
    data.FIELD_22 != 1
]  # exclude people who are on medication for high blood pressure

data = data[data.FIELD_17 != 1]  # exclude people who are diagnosed for hyperlipidemia
data = data[
    data.FIELD_24 != 1
]  # exclude people who are on medication for hyperlipidemia

print(data.shape)

data = data[
    [
        "L100800",
        "L104600",
        "L103000",
        "S000300",
        "L101700",
        "L100700",
        "FIELD_33",
        "FIELD_38",
        "FIELD_40",
        "FIELD_31",
        "SEX",
        "AGE",
        "CLASS",
    ]
]
data = data.dropna()
print(data.shape)
data.head()



(185843, 19)
(185843, 2)
(169024, 20)
(56542, 13)


,L100800,L104600,L103000,S000300,L101700,L100700,FIELD_33,FIELD_38,FIELD_40,FIELD_31,SEX,AGE,CLASS
2,78.0,5.28,41.0,20.2,15.0,3.8,1.0,2.0,1.0,0.0,1.0,46.0,0
5,90.0,5.74,50.0,25.5,12.0,3.4,1.0,0.0,1.0,0.0,1.0,52.0,0
10,86.0,5.83,45.0,21.2,17.0,3.9,1.0,0.0,1.0,1.0,1.0,37.0,0
11,86.0,4.73,54.0,22.0,30.0,4.2,1.0,2.0,3.0,0.0,1.0,39.0,0
20,87.0,5.60,340.0,24.6,26.0,4.7,1.0,0.0,2.0,0.0,1.0,59.0,0


<IPython.core.display.Javascript object>

In [12]:
diabetic = data[data.CLASS == 2]
prediabetic = data[data.CLASS == 1]
normal = data[data.CLASS == 0]

print(diabetic.shape[0], prediabetic.shape[0], normal.shape[0])

1045 17331 38166


<IPython.core.display.Javascript object>

In [13]:
diabetic_test = diabetic.sample(200, random_state=randomseed)
prediabetic_test = prediabetic.sample(200, random_state=randomseed)
normal_test = normal.sample(200, random_state=randomseed)
test = pd.concat([diabetic_test, prediabetic_test, normal_test])

diabetic_train = diabetic.drop(diabetic_test.index)
prediabetic_train = prediabetic.drop(prediabetic_test.index)
# .sample(
#     10 * diabetic_train.shape[0], random_state=randomseed
# )
normal_train = normal.drop(normal_test.index).sample(
    prediabetic_train.shape[0],
    random_state=randomseed
    #     10 * diabetic_train.shape[0], random_state=randomseed
)
train = pd.concat([diabetic_train, diabetic_train, prediabetic_train, normal_train])

<IPython.core.display.Javascript object>

In [14]:
xtrain = train.iloc[:, :-1]
ytrain = train.iloc[:, -1]
xtest = test.iloc[:, :-1]
ytest = test.iloc[:, -1]

<IPython.core.display.Javascript object>

In [15]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

<IPython.core.display.Javascript object>

In [16]:
from imblearn.over_sampling import SMOTE, SMOTENC  # doctest: +NORMALIZE_WHITESPACE

randomseed = 42

sm = SMOTE(random_state=randomseed, sampling_strategy="minority")
X_res, y_res = sm.fit_resample(xtrain, ytrain)

print("Resampled dataset shape %s" % Counter(y_res))
print(
    y_res[y_res == 0].shape[0], y_res[y_res == 1].shape[0], y_res[y_res == 2].shape[0]
)
print(X_res.shape, y_res.shape)

xtrain = X_res
ytrain = y_res

Resampled dataset shape Counter({2: 17131, 1: 17131, 0: 17131})
17131 17131 17131
(51393, 12) (51393,)


<IPython.core.display.Javascript object>

3.Train and Generate models 
===

In [17]:
weakmodles = []
estimators = []


<IPython.core.display.Javascript object>

In [18]:
def changelist(strlist):
    test_list = (strlist).replace("[", "").replace("]", "").replace("'", "").split(",")
    test_list = [int(i) for i in test_list]
    return test_list

<IPython.core.display.Javascript object>

In [19]:
rf_12 = RandomForestClassifier(
    random_state=randomseed,
    n_estimators=100,
    max_depth=12,
    min_samples_split=2,
    min_samples_leaf=10,
    max_features="auto",
)

xgb_model_12 = xgb.XGBClassifier(objective="binary:logistic", random_state=randomseed)

scv_12 = svmgpu(
    C=100,
    cache_size=None,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=True,
    random_state=None,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

<IPython.core.display.Javascript object>

In [20]:
weakmodles = []
estimators = []

for i in range(25):
    if modelslist.iloc[i, -1] == "RF":

        weakmodles.append(
            make_pipeline(ColumnSelector(cols=changelist(modelslist.iloc[i, 0])), rf_12)
        )
        estimators.append(
            (
                (
                    "rf_12" + str(i),
                    (
                        make_pipeline(
                            ColumnSelector(cols=changelist(modelslist.iloc[i, 0])),
                            rf_12,
                        )
                    ),
                )
            )
        )
    elif modelslist.iloc[i, -1] == "SVM":

        weakmodles.append(
            make_pipeline(
                ColumnSelector(cols=changelist(modelslist.iloc[i, 0])), scv_12
            )
        )
        estimators.append(
            (
                (
                    "scv_12" + str(i),
                    (
                        make_pipeline(
                            ColumnSelector(cols=changelist(modelslist.iloc[i, 0])),
                            scv_12,
                        )
                    ),
                )
            )
        )

    elif modelslist.iloc[i, -1] == "XGB":

        weakmodles.append(
            make_pipeline(
                ColumnSelector(cols=changelist(modelslist.iloc[i, 0])), xgb_model_12
            )
        )
        estimators.append(
            (
                (
                    "xgb_model_12" + str(i),
                    (
                        make_pipeline(
                            ColumnSelector(cols=changelist(modelslist.iloc[i, 0])),
                            xgb_model_12,
                        )
                    ),
                )
            )
        )

<IPython.core.display.Javascript object>

In [21]:
# for i in range(len(Rf_list)):
#     weakmodles.append(
#         make_pipeline(ColumnSelector(cols=changelist(Rf_list.iloc[i, 0])), rf_12)
#     )
#     estimators.append(
#         (
#             (
#                 "rf_12" + str(i),
#                 (
#                     make_pipeline(
#                         ColumnSelector(cols=changelist(Rf_list.iloc[i, 0])), rf_12
#                     )
#                 ),
#             )
#         )
#     )

<IPython.core.display.Javascript object>

In [22]:
# for i in range(len(xgb_list)):
#     weakmodles.append(
#         make_pipeline(
#             ColumnSelector(cols=changelist(xgb_list.iloc[i, 0])), xgb_model_12
#         )
#     )
#     estimators.append(
#         (
#             (
#                 "xgb_model_12" + str(i),
#                 (
#                     make_pipeline(
#                         ColumnSelector(cols=changelist(xgb_list.iloc[i, 0])),
#                         xgb_model_12,
#                     )
#                 ),
#             )
#         )
#     )

<IPython.core.display.Javascript object>

In [23]:
# for i in range(len(svm_list)):
#     weakmodles.append(
#         make_pipeline(ColumnSelector(cols=changelist(svm_list.iloc[i, 0])), scv_12)
#     )
#     estimators.append(
#         (
#             (
#                 "scv_12" + str(i),
#                 (
#                     make_pipeline(
#                         ColumnSelector(cols=changelist(svm_list.iloc[i, 0])), scv_12
#                     )
#                 ),
#             )
#         )
#     )

<IPython.core.display.Javascript object>

4.Train ensemble modles
===

In [24]:
acclist = []
predprobalist = []
predprobalist2 = []
confmatrxlist = []
ypredconfprob_all = []

<IPython.core.display.Javascript object>

In [25]:
counter = 0
for f in weakmodles:

    print(counter)
    counter = counter + 1

    rfclf = f
    rfclf.fit(xtrain, ytrain)
    ypred = rfclf.predict(xtest)
    ypredprob = rfclf.predict_proba(xtest)

    accuracy = m.accuracy_score(ytest, ypred)
    acclist.append(accuracy)
    predprobalist.append(ypredprob)
    predprobalist2.append(ypred)

    confmat = m.confusion_matrix(ytest, ypred)
    confmatrxlist.append(confmat)

    confsumh = np.sum(confmat, axis=1)
    propconfmat = confmat.copy()
    for i in range(propconfmat.shape[0]):
        propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
    ypredconfprob_all.append(propconfmat / 100)


cimc2 = np.zeros((ytest.shape[0], 3))
# # for each classifier
for j in range(len(weakmodles)):
    #     #for each data point
    cimc = []
    for i in range(ytest.shape[0]):
        # print(i,ypredconfprob_all[j],predprobalist[j][i])

        c1 = np.sum(predprobalist[j][i] * ypredconfprob_all[j][0])
        c2 = np.sum(predprobalist[j][i] * ypredconfprob_all[j][1])
        c3 = np.sum(predprobalist[j][i] * ypredconfprob_all[j][2])

        # print([c1,c2,c3])
        cimc.append([c1, c2, c3])

    cimc = np.array(cimc)
    ypred2 = np.argmax(cimc, axis=1)
    print((m.accuracy_score(ytest, ypred2)))
    cimc2 = cimc2 + cimc

print("final cim ==== >")
cimc2 = np.array(cimc2)
ypred3 = np.argmax(cimc2, axis=1)
print((m.accuracy_score(ytest, ypred3)))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
0.7333333333333333
0.7283333333333334
0.7283333333333334
0.7366666666666667
0.7366666666666667
0.7383333333333333
0.7333333333333333
0.7266666666666667
0.7216666666666667
0.7266666666666667
0.74
0.7433333333333333
0.74
0.7283333333333334
0.7366666666666667
0.7316666666666667
0.7216666666666667
0.725
0.725
0.7333333333333333
0.7366666666666667
0.7366666666666667
0.74
0.72
0.7233333333333334
final cim ==== >
0.7366666666666667


<IPython.core.display.Javascript object>

In [26]:
print("final cim ==== >")
cimc2 = np.array(cimc2)
ypred3 = np.argmax(cimc2, axis=1)
print((m.accuracy_score(ytest, ypred3)))

final cim ==== >
0.7366666666666667


<IPython.core.display.Javascript object>

cross val
===

In [27]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, random_state=randomseed, shuffle=True)
kf.get_n_splits(xtrain)

print(kf)

data_cv = []
cv_acc = []

counter = 0
for train_index, test_index in kf.split(xtrain):
    # print("TRAIN:", train_index, "TEST:", test_index)
    cv_xtrain, cv_xtest = xtrain[train_index], xtrain[test_index]
    cv_ytrain, cv_ytest = ytrain[train_index], ytrain[test_index]

    acclist = []
    predprobalist = []
    predprobalist2 = []
    confmatrxlist = []
    ypredconfprob_all = []

    print(counter)
    counter = counter + 1

    for f in weakmodles:

        rfclf = f
        rfclf.fit(cv_xtrain, cv_ytrain)
        ypred = rfclf.predict(cv_xtest)
        ypredprob = rfclf.predict_proba(cv_xtest)

        accuracy = m.accuracy_score(cv_ytest, ypred)
        acclist.append(accuracy)
        predprobalist.append(ypredprob)
        predprobalist2.append(ypred)

        confmat = m.confusion_matrix(cv_ytest, ypred)
        confmatrxlist.append(confmat)

        confsumh = np.sum(confmat, axis=1)
        propconfmat = confmat.copy()
        for i in range(propconfmat.shape[0]):
            propconfmat[i] = 100 * propconfmat[i] / confsumh[i]
        ypredconfprob_all.append(propconfmat / 100)

    cimc2 = np.zeros((cv_ytest.shape[0], 3))
    # # for each classifier
    for j in range(len(weakmodles)):
        #     #for each data point
        cimc = []
        for i in range(cv_ytest.shape[0]):
            # print(i,ypredconfprob_all[j],predprobalist[j][i])

            c1 = np.sum(predprobalist[j][i] * ypredconfprob_all[j][0])
            c2 = np.sum(predprobalist[j][i] * ypredconfprob_all[j][1])
            c3 = np.sum(predprobalist[j][i] * ypredconfprob_all[j][2])

            # print([c1,c2,c3])
            cimc.append([c1, c2, c3])

        cimc = np.array(cimc)
        ypred2 = np.argmax(cimc, axis=1)
        print((m.accuracy_score(cv_ytest, ypred2)))
        cimc2 = cimc2 + cimc

    print("final cim ==== >")
    cimc2 = np.array(cimc2)
    ypred3 = np.argmax(cimc2, axis=1)
    print((m.accuracy_score(cv_ytest, ypred3)))
    cv_acc.append((m.accuracy_score(cv_ytest, ypred3)))

KFold(n_splits=10, random_state=42, shuffle=True)
0
0.7494163424124514
0.7589494163424124
0.7556420233463035
0.7367704280155642
0.7490272373540856
0.7480544747081712
0.7533073929961089
0.7513618677042802
0.7624513618677042
0.7581712062256809
0.7375486381322958
0.7381322957198444
0.7383268482490273
0.7365758754863814
0.7542801556420233
0.7441634241245136
0.7599221789883268
0.7589494163424124
0.7554474708171206
0.7570038910505836
0.7379377431906615
0.7538910505836576
0.7476653696498055
0.7624513618677042
0.7587548638132295
final cim ==== >
0.7542801556420233
1
0.7540856031128405
0.761284046692607
0.7542801556420233
0.7307392996108949
0.7544747081712062
0.7464980544747082
0.7573929961089494
0.7538910505836576
0.7667315175097277
0.7605058365758754
0.7315175097276264
0.7319066147859922
0.7340466926070038
0.7328793774319066
0.7476653696498055
0.7517509727626459
0.7558365758754864
0.7571984435797665
0.7494163424124514
0.7595330739299611
0.7297665369649805
0.7531128404669261
0.7540856031128405

<IPython.core.display.Javascript object>

In [28]:
# top 25 new
print(np.mean(cv_acc))
print(np.std(cv_acc))

0.7599479943939796
0.004902541241269364


<IPython.core.display.Javascript object>

In [101]:
# top 20
print(np.mean(cv_acc))
print(np.std(cv_acc))

0.7612905242053027
0.0053572377800363506


<IPython.core.display.Javascript object>

In [84]:
# top 40
print(np.mean(cv_acc))
print(np.std(cv_acc))

0.7625359140410215
0.00474405028380742


<IPython.core.display.Javascript object>

In [79]:
# top 25
print(np.mean(cv_acc))
print(np.std(cv_acc))

0.7676143975686045
0.004732690071998157


<IPython.core.display.Javascript object>

In [77]:
# top 10
print(np.mean(cv_acc))
print(np.std(cv_acc))

0.7645205807728039
0.004473010776066563


<IPython.core.display.Javascript object>